In [ ]:
from analyse_trips import *
import geopandas as gpd
import shapely as shp
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import folium
from folium import plugins

In [ ]:
gdf = load_data("mapstr_2024_04_24.geojson")
#   https://www.youtube.com/watch?v=xu1wRfUHtKg 
plot_interactive_map(gdf)

In [ ]:
gdf.head()

In [ ]:
# drop column a
gdf.drop(columns="userComment", inplace=True)
gdf.drop(columns="icon", inplace=True)
gdf.head()

In [ ]:
unpack_tag(gdf["tags"])

In [ ]:
plot_interactive_map(load_data(FRENCH_DEPARTEMENTS))

In [ ]:
plot_interactive_map(load_data(WORLD_COUNTRIES))


In [ ]:
plot_interactive_map(load_data("data/german_lander.geojson"))

In [ ]:
world_countries_gdf = load_data(WORLD_COUNTRIES)
world_countries_gdf

In [ ]:
joined = gpd.sjoin(gdf, world_countries_gdf, how="inner", predicate='intersects')
print(len(set(joined["french_short"])))
joined

In [ ]:
gdf["country"] = joined["name_right"]
gdf["country_FR"] = joined["french_short"]
gdf["continent"] = joined["continent"]
gdf


In [ ]:
departements_gdf = load_data(FRENCH_DEPARTEMENTS)
departements_gdf

In [ ]:
joined = gpd.sjoin(gdf, departements_gdf, how="inner", predicate='intersects')
joined

In [ ]:
gdf["departement"] = joined["nom"]
gdf

In [ ]:
unpack_tag(gdf["tags"])

In [ ]:
plot_interactive_map(load_data(FRENCH_DEPARTEMENTS))

In [ ]:
plot_interactive_map(load_data(WORLD_COUNTRIES))


In [ ]:
world_countries_gdf = load_data(WORLD_COUNTRIES)
world_countries_gdf

In [ ]:
joined = gpd.sjoin(gdf, world_countries_gdf, how="inner", predicate='intersects')
print(len(set(joined["french_short"])))
joined

In [ ]:
gdf["country"] = joined["name_right"]
gdf["country_FR"] = joined["french_short"]
gdf["continent"] = joined["continent"]
gdf


In [ ]:
departements_gdf = load_data(FRENCH_DEPARTEMENTS)
departements_gdf

In [ ]:
joined = gpd.sjoin(gdf, departements_gdf, how="inner", predicate='intersects')
joined

In [ ]:
gdf["departement"] = joined["nom"]
gdf

In [ ]:
visited_departements = set(gdf["departement"])
visited_departements.remove(np.nan)
len(visited_departements)

In [ ]:
gdf = gdf.to_crs("EPSG:4087")
world_countries_gdf = world_countries_gdf.to_crs("EPSG:4087")

set(gdf.distance(world_countries_gdf.unary_union))

In [ ]:
world_union = world_countries_gdf.unary_union

nearest_countries = []
for point in gdf["geometry"]:
    nearest_geom = shp.ops.nearest_points(point, world_union)[1]
    nearest_country = world_countries_gdf[
        world_countries_gdf["geometry"] == nearest_geom
    ]#["name"].values[0]
    
    nearest_countries.append(nearest_country)
    
nearest_countries

In [ ]:
from shapely import STRtree

tree = STRtree(world_countries_gdf["geometry"])
nearest_countries = []
for point in gdf["geometry"]:
    # point = gdf["geometry"].iloc[249]
    nearest_geom = tree.nearest(point)
    # break
    nearest_country = world_countries_gdf.loc[nearest_geom]

    nearest_countries.append(nearest_country)
# world_countries_gdf.loc[nearest_geom]
country = gpd.GeoDataFrame(nearest_countries, index=gdf.index)


## Graphical representations with Folium

In [ ]:
gdf = clean_mapstr_data(load_data("mapstr_2024_04_24.geojson"))

In [ ]:
gdf

In [ ]:

my_map = folium.Map(location=[15, -20], tiles="Cartodb dark_matter", zoom_start=2)


heat_data = [[point.xy[1][0], point.xy[0][0]] for point in gdf.to_crs(crs=4326).geometry]

heat_data
plugins.HeatMap(heat_data).add_to(my_map)

my_map

In [ ]:
gdf_france = gdf[gdf["country"] == "France"]
gdf_france.explore()

In [ ]:
departements_dict = {}
gdf_france = gdf[gdf["country"] == "France"]
departements_gdf = load_data(FRENCH_DEPARTEMENTS)


for dep in gdf_france["departement"]:
    if dep is not np.nan:
        try:
            departements_dict[dep] += 1
        except KeyError:
            departements_dict[dep] = 1
# departements_dict = set(gdf_france["departement"])
# departements_dict.remove(np.nan)
gdf_departements = departements_gdf[departements_gdf["nom"].isin(departements_dict.keys())]
gdf_departements["count"] = [departements_dict[dep] for dep in gdf_departements["nom"]]
gdf_departements.explore("count", scheme="natural_breaks", k=11, cmap="autumn_r")

In [ ]:
# folium plot of the visited departements
my_map = folium.Map(location=[47.1, 2.4], zoom_start=6)
# add all departements polygons
for i, row in gdf_departements.iterrows():
    folium.GeoJson(row["geometry"]).add_to(my_map)
my_map

In [ ]:
gdf_us = gdf[gdf["country"] == "United States of America"]
us_states_gdf = load_data("data/us_states.geojson")

us_states_dict = {}
for dep in gdf_us["US_state"]:
    if dep is not np.nan:
        try:
            us_states_dict[dep] += 1
        except KeyError:
            us_states_dict[dep] = 1

gdf_us_states_visited = us_states_gdf[us_states_gdf["NAME"].isin(us_states_dict.keys())]
gdf_us_states_visited["count"] = [us_states_dict[dep] for dep in gdf_us_states_visited["NAME"]]
gdf_us_states_visited.explore("count", scheme="natural_breaks", k=9, cmap="viridis_r")

In [ ]:
gdf_DE = gdf[gdf["country"] == "Germany"]
lander_gdf = load_data("data/german_lander.geojson")

landers_dict = {}
for dep in gdf_DE["DE_lander"]:
    if dep is not np.nan:
        try:
            landers_dict[dep] += 1
        except KeyError:
            landers_dict[dep] = 1

gdf_lander_visited = lander_gdf[lander_gdf["lander_name"].isin(landers_dict.keys())]
gdf_lander_visited["count"] = [landers_dict[dep] for dep in gdf_lander_visited["lander_name"]]
gdf_lander_visited.explore("count", scheme="natural_breaks", cmap="autumn_r")

In [ ]:
# Italia y espana

## Rivers

In [ ]:
plot_interactive_map(load_data("data/rivers.geojson").to_crs(epsg=4087).buffer(1000).to_crs(epsg=4326))

In [ ]:
plot_interactive_map(load_data("data/lakes.geojson"))

In [ ]:
plot_interactive_map(load_data("data/oceans.geojson"))

In [ ]:
rivers = load_data("data/rivers.geojson").to_crs(epsg=4087)#.buffer(1000)
oceans = load_data("data/oceans.geojson").to_crs(epsg=4087)
lakes = load_data("data/lakes.geojson").to_crs(epsg=4087)
gdf = clean_mapstr_data(load_data("mapstr_2024_04_24.geojson")).to_crs(epsg=4087)

In [ ]:
close_to_rivers = gdf.distance(rivers.unary_union) < 2000
close_to_oceans = gdf.distance(oceans.unary_union) < 2000
close_to_lakes = gdf.distance(lakes.unary_union) < 2000
gdf_water = gdf[close_to_rivers | close_to_oceans | close_to_lakes]
print(gdf_water.shape)
gdf_water.explore()